In [1]:
import numpy as np

from fashion_mnist_cnn import *

In [2]:
def selection(pop, fitness, num_parents):
    # We select the ones with the highest fitness
    sorted_indecies = np.argsort(fitness)[::-1] # we reverse array, because we want ascending
    top_indecies = sorted_indecies[:num_parents]
    parents = pop[top_indecies]
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty((offspring_size, parents.shape[1]), dtype=np.int)
    # The point at which crossover takes place between two parents
    crossover_part = int(parents.shape[1]/2)
    # Define all the offsprings
    for k in range(0, offspring_size, 2):
        # we assign random from the two parents
        random_choice = np.random.choice(parents.shape[1], crossover_part, replace=False)
        gene_selection = np.zeros(parents.shape[1])
        gene_selection[random_choice] = 1
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # Assign the correct parts of the parents to the offspring
        offspring[k, np.where(gene_selection == 0)[0]] = parents[parent1_idx, np.where(gene_selection == 0)[0]]
        offspring[k, np.where(gene_selection == 1)[0]] = parents[parent2_idx, np.where(gene_selection == 1)[0]]
        offspring[k+1, np.where(gene_selection == 1)[0]] = parents[parent1_idx, np.where(gene_selection == 1)[0]]
        offspring[k+1, np.where(gene_selection == 0)[0]] = parents[parent2_idx, np.where(gene_selection == 0)[0]]
    return offspring

def mutation(offspring_crossover, r_mut=0.2):
    # Do if mutation stikes, must be changed if number of options exceed 2
    for idx in range(offspring_crossover.shape[0]):
        for gene_idx in range(offspring_crossover.shape[1]): 
            # Flip gene if mutation happens
            if (np.random.rand() < r_mut): 
                offspring_crossover[idx, gene_idx] = 1-offspring_crossover[idx, gene_idx]# bit flip
    return offspring_crossover

In [3]:
"""
The target is to maximize the accuracy of the cnn-mnist model:
    We start by having a few decisions related to the model. 
    // Model hyper-parameters
    x1 = (32) or 64 or 128 filters in each CNN layer
    x2 = 50 or 100 neurons in first dense layer
    x3 = 1 or 2 CNN layers
    x4 = he_uniform or random_normal
    x5 = no dropout, (0.1 dropout,) 0.2 dropout
    x6 = learning rate 0.005, (0.01) or 0.05 
    x7 = kernel size cnn (2,) 3, 4
    We will use a genetic algorithm to find the best combinations of these hyperparameters. 
    The fitness-function is based on model accuracy on test-dataset. 
"""
num_weights = 7
# how many options a hyperparameter has
options = [2, 2, 2, 2, 2, 2, 2]

# solutions per populations and number of mating parents
sol_per_pop = 6
num_parents = 3

In [4]:
# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
new_population = np.random.randint(low=0, high=2, size=pop_size)
print(new_population)

[[0 1 1 0 1 1 1]
 [1 0 0 0 1 1 0]
 [1 0 1 1 0 1 0]
 [0 0 1 0 0 1 1]
 [0 1 0 0 0 1 0]
 [1 1 0 0 1 0 0]]


In [5]:
best_outputs = []
num_generations = 5

In [6]:
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = evaluate_model_ga(new_population)
    print("Fitness")
    print(fitness)
    
    # Find best result from gen x
    generation_best = (np.max(fitness), new_population[np.where(fitness == np.max(fitness))[0]])
    best_outputs.append(generation_best)
    # The best result in the current iteration.
    print("Best result : ", generation_best)
    
    # Selecting the best parents in the population for mating.
    parents = selection(new_population, fitness, num_parents)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, sol_per_pop)
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, r_mut=0.2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population = offspring_mutation

Generation :  0
Epoch 1/30
1500/1500 [==============================] - 24s 15ms/step - loss: 1.3082 - accuracy: 0.5101 - val_loss: 0.8315 - val_accuracy: 0.6842
Epoch 2/30
1500/1500 [==============================] - 20s 14ms/step - loss: 0.8427 - accuracy: 0.6809 - val_loss: 0.7573 - val_accuracy: 0.7086
Epoch 3/30
1500/1500 [==============================] - 20s 14ms/step - loss: 0.7784 - accuracy: 0.7068 - val_loss: 0.7238 - val_accuracy: 0.7239
Epoch 4/30
1500/1500 [==============================] - 22s 14ms/step - loss: 0.7439 - accuracy: 0.7143 - val_loss: 0.6760 - val_accuracy: 0.7398
Epoch 5/30
1500/1500 [==============================] - 22s 14ms/step - loss: 0.7221 - accuracy: 0.7234 - val_loss: 0.6786 - val_accuracy: 0.7312
Epoch 6/30
1500/1500 [==============================] - 21s 14ms/step - loss: 0.7075 - accuracy: 0.7325 - val_loss: 0.6089 - val_accuracy: 0.7584
Epoch 7/30
1500/1500 [==============================] - 21s 14ms/step - loss: 0.6891 - accuracy: 0.7385 - va

KeyboardInterrupt: 